There are two common sets of APIs to load a Keras model, a high-level API and a low-level API. This tutorial shows you how to load a Keras model using the SavedModel API under a distributed strategy.

# Environment Settings

In [ ]:
# !apt-get update && apt-get install -y iputils-ping net-tools

In [ ]:
# !pip install -q tensorflow_datasets
!pip install -q tensorflow_hub

In [24]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

tfds.disable_progress_bar()

print("Tensorflow Version: {}".format(tf.__version__))
print("Eager Mode: {}".format(tf.executing_eagerly()))
print("GPU {} available.".format("is" if tf.config.experimental.list_physical_devices("GPU") else "not"))

Tensorflow Version: 2.1.0
Eager Mode: True
GPU not available.


# Training with a Distributed Strategy

In [2]:
accelerators = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [3]:
def get_data():
  datasets, info = tfds.load(
    name="fashion_mnist", with_info=True, as_supervised=True)
  fashion_train, fashion_test = datasets['train'], datasets['test']
    
  BUFFER_SIZE = 10000
  BATCH_SIZE_PER_REPLICA = 64
  BATCH_SIZE = BATCH_SIZE_PER_REPLICA * accelerators.num_replicas_in_sync

  def normalize(image, label):
    img = tf.cast(image, tf.float32)
    img /= 255.0
    return img, label

  train_dataset = fashion_train.map(normalize).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
  test_dataset = fashion_test.map(normalize).cache().batch(BATCH_SIZE)

  return train_dataset, test_dataset

In [10]:
def build_model():
  def _model_body(inputs):
    x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='elu')(inputs)
    x = tf.keras.layers.MaxPool2D()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(units=64, activation='elu')(x)
    output = tf.keras.layers.Dense(units=10, activation='softmax')(x)
    return output
  
  inputs = tf.keras.Input(shape=(28, 28, 1))
  outputs = _model_body(inputs)
  model = tf.keras.Model(inputs, outputs)
  return model

def get_model():
  with accelerators.scope():
    model = build_model()
    model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, 
                  optimizer=tf.keras.optimizers.Adam(), 
                  metrics=['accuracy'])
  return model

Download and preprocess the datasets.

In [5]:
train_dataset, eval_dataset = get_data()

In [6]:
for _image, _label in train_dataset.take(1):
  print(_image.shape, _label.shape)

(64, 28, 28, 1) (64,)


Train a model using a distributed strategy.

In [11]:
model = get_model()
model.fit(train_dataset, epochs=2)

Epoch 1/2
938/938 [==============================] - 44s 47ms/step - loss: 0.4378 - accuracy: 0.8456
Epoch 2/2
938/938 [==============================] - 23s 25ms/step - loss: 0.3026 - accuracy: 0.8913


# Save and Load the Model

Two common APIs are available for loading a `TF.Keras` model.
* a high-level API: `tf.keras.Model.save` (or `model.save`) and `tf.keras.models.load_model`.
* a low-level API: `tf.saved_model.save` and `tf.saved_model.load`

## The TF.Keras APIs

### Saving the Model

You must save the model without the scope.

In [12]:
keras_model_path = "/tmp/keras_save"
model.save(keras_model_path)   # save() should be called out of the scope

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: /tmp/keras_save/assets


INFO:tensorflow:Assets written to: /tmp/keras_save/assets


### Loading the Model without the Scope

In [13]:
restored_keras_model = tf.keras.models.load_model(keras_model_path)
restored_keras_model.fit(train_dataset, epochs=2)

Epoch 1/2
938/938 [==============================] - 40s 42ms/step - loss: 0.2595 - accuracy: 0.9061
Epoch 2/2
938/938 [==============================] - 22s 24ms/step - loss: 0.2199 - accuracy: 0.9205


After loading the model, you can continue to train the model without compiling it again. The model is compiled before saving it and is saved as a Tensorflow standard proto format.

### Loading the Model within the Specific Scope

In [14]:
specific_scope = tf.distribute.OneDeviceStrategy('/cpu:0')
with specific_scope.scope():
  restored_keras_model_ds = tf.keras.models.load_model(keras_model_path)
  restored_keras_model_ds.fit(train_dataset, epochs=2)

Epoch 1/2
938/938 [==============================] - 46s 50ms/step - loss: 0.2593 - accuracy: 0.9068
Epoch 2/2
938/938 [==============================] - 27s 29ms/step - loss: 0.2216 - accuracy: 0.91941s - l


**Here as you can see, you can restore and train the model on a different scope from the saved one.**

## The `tf.saved_model` APIs

### Save the Model

In [15]:
model = get_model()
saved_model_path = "/tmp/tf_saved"
tf.saved_model.save(model, saved_model_path)

INFO:tensorflow:Assets written to: /tmp/tf_saved/assets


INFO:tensorflow:Assets written to: /tmp/tf_saved/assets


### Load the Model without the Scope

Loading the model using the `tf.saved_model` APIs provides low-level functionality (but as a foundation for lots of use cases) and returns an object, not a `TF.Keras` model. This object contains a function allowing to do inference.

In [16]:
DEFAULT_FUNCTION_KEY = 'serving_default'
loaded = tf.saved_model.load(saved_model_path)
inference_func = loaded.signatures[DEFAULT_FUNCTION_KEY]

Each function is associated with a key. The `serving_default` is the default key for the inference function. To do the inference with the above function.

In [ ]:
predict_dataset = eval_dataset.map(lambda image, label: image)
for batch in predict_dataset.take(1):
  print(inference_func(batch))

### Loading the Model under a Distributed Manner

In [ ]:
distributed = tf.distribute.MirroredStrategy()

with distributed.scope():
  loaded = tf.saved_model.load(saved_model_path)
  inference_func = loaded.signatures[DEFAULT_FUNCTION_KEY]
    
  # define the dataset allowing for a distributed manner
  dist_predict_dataset = distributed.experimental_distribute_dataset(predict_dataset)
  
  # calling the function in a distributed manner
  for batch in dist_predict_dataset:
    pred_res = distributed.experimental_run_v2(inference_func, args=(batch,))

pred_res

Calling the inference function is the forward pass on the saved model, as a prediction operation. However, if you are going to continue training or to embed the loaded model into a bigger one, you need a further operation. A common way is to wrap this loaded object into a Keras layer to achieve the above goals.

In [27]:
def build_bigger_model(loaded):
  x = tf.keras.Input(shape=(28, 28, 1), name='input')
  
  # wrap the loaded model to a KerasLayer
  keras_layer = hub.KerasLayer(loaded, trainable=True)(x)
    
  model = tf.keras.Model(x, keras_layer)
  return model

In [28]:
accel_strategy = tf.distribute.MirroredStrategy()

with accel_strategy.scope():
  loaded = tf.saved_model.load(saved_model_path)
  model = build_bigger_model(loaded)

  model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, 
                optimizer=tf.keras.optimizers.Adam(), 
                metrics=['accuracy'])
  model.fit(train_dataset, epochs=2)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


Epoch 1/2
938/938 [==============================] - 43s 46ms/step - loss: 0.4259 - accuracy: 0.8482
Epoch 2/2
938/938 [==============================] - 23s 24ms/step - loss: 0.2949 - accuracy: 0.8947


As you can see, you can wrap the loaded object into a bigger model or an another model using the `hub.KerasLayer` APIs. Such operations are useful for transfer learning.

## Choices

* about `Saving` the model: It is recommended to save the model using `model.save`. However, if your model is not constructed on the Keras APIs, you can save the model using the lower-level APIs.

* about `Loading` the model: It depends on the purpose of what you want to do. If you want to get a Keras model, the `tf.keras.models.load_model()` API is suitable for you, on the other hand, if you want to deploy the model and do the inference task, the low-level API `tf.saved_model.load()` might be better for you.

You can also mix these two different APIs at all.

In [30]:
model = get_model()

# save the model using the high-level API
model.save(keras_model_path)

# load the model using the lower-level (not tf.keras) API
with accel_strategy.scope():
  loaded_model = tf.saved_model.load(keras_model_path)

INFO:tensorflow:Assets written to: /tmp/keras_save/assets


INFO:tensorflow:Assets written to: /tmp/keras_save/assets


## Caveats

**You can notice there is a special case that a Keras model does not have well-defined inputs.** For example, a sequential model can be created without a well-defined input. This comes with an error when the subclass module does not initialize the input as well. **In such a case, you have to stick with the low-level APIs on both saving and loading the model, otherwise, the model comes with errors.**

It is easy to check the model whether it has well-defined inputs. Call the attribute `model.inputs` to check it is `None` or not. In general, the model's input would be defined after the model is used in `.fit()`, `.eval()`, `.predict()` or when calling `model(inputs)`.

In [31]:
class ExampleModel(tf.keras.Model):
  
  output_layer_name = "output_layer"
    
  def __init__(self):
    super(ExampleModel, self).__init__()
    self._dense_layer = tf.keras.layers.Dense(
      units=5, dtype=tf.float32, name=self.output_layer_name)
    
  def call(self, inputs):
    return self._dense_layer(inputs)

examplemodel = ExampleModel()

In [36]:
example_path = "/tmp/example"

if len(examplemodel.inputs) < 1:
  try:
    examplemodel.save(example_path)
    print("Call a low-level API `model.save()`.")
  except Exception as e:
    print("Failed in `model.save().` {}".format(e))
    tf.saved_model.save(examplemodel, example_path)
    print("Call a high-level API `tf.saved_model.save()`.")

Failed in `model.save().` Model <__main__.ExampleModel object at 0x7f6ac54e2cf8> cannot be saved because the input shapes have not been set. Usually, input shapes are automatically determined from calling .fit() or .predict(). To manually set the shapes, call model._set_inputs(inputs).


INFO:tensorflow:Assets written to: /tmp/example/assets


INFO:tensorflow:Assets written to: /tmp/example/assets


Call a high-level API `tf.saved_model.save()`.
